## Download and extract as separate texts all posts in an section of choice from <http://forum.lvivport.com>

In [1]:
import requests, re
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm
from zipfile import ZipFile

In [2]:
topics = []

for page in tqdm(range(1, 12)): # 11 pages in section
    setion_url = f'http://forum.lvivport.com/forums/kimnata-smixu.37/page-{page}'
    page_html = requests.get(setion_url).text
    soup = BeautifulSoup(page_html, 'lxml')
    topics += [a.get('href') for a in soup.select('.discussionListItems li h3 > a')]
    sleep(1)

100%|██████████| 11/11 [00:15<00:00,  1.37s/it]


In [3]:
def get_post_data(post, topic):
    return {
        'author': post.select('.username.author')[0].get('href').split('/')[-2],
        'date': post.select('.DateTime')[0].text,
        'text': post.select('blockquote.messageText')[0].text.strip(),
        'topic': topic
    }

In [ ]:
all_posts = []
for topic in tqdm(topics):
    thread_html = requests.get(f'http://forum.lvivport.com/{topic}').text
    thread_soup = BeautifulSoup(thread_html, 'lxml')
    try:
        pagerange = int(re.findall('\d+', thread_soup.select('.pageNavHeader')[0].text)[-1]) + 1
    except IndexError: # if single-page thread
        pagerange = 2
    
    for page in range(1, pagerange):
        thr_page_html = requests.get(f'http://forum.lvivport.com/{topics[5]}page-{page}')
        soup = BeautifulSoup(thr_page_html.text, 'lxml')
        all_posts += [get_post_data(post, topic)
                      for post in soup.select('div.messageInfo.primaryContent')]
        
        sleep(1)

In [23]:
with ZipFile('2-2-forum_humor.zip', 'w') as posts_zip:
    for i, post in enumerate(all_posts):
        topic_id = re.search('.*\.(\d+)/', post['topic']).group(1)
        with posts_zip.open(f'{i}_{topic_id}_{post["author"]}_{post["date"].replace(" ", "-")}.txt', 'w') as f:
            f.write(bytes(post['text'], encoding='utf-8'))